In [9]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model
from tensorflow.keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
import tensorflow as tf

In [10]:
X_train = load('trainXacc.npy')
X_test = load('testXacc.npy')
y_train = load('trainYacc.npy')
y_test = load('testYacc.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(33104, 128, 3) (33104, 118) (3740, 128, 3) (3740, 118)


In [11]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [12]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [13]:
model1 = Sequential()
model1.add(LSTM(512, input_shape=(n_timesteps,n_features)))
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(0.1))
model1.add(Dense(128, activation='relu'))
model1.add(Dense(n_outputs, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()
callbacks = [tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 512)               1056768   
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 118)               15222     
Total params: 1,236,214
Trainable params: 1,236,214
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model1.fit(X_training, y_training, epochs=200, verbose=True, validation_data=(X_validation, y_validation), batch_size=128,callbacks=callbacks)

Epoch 1/200
207/207 [==============================] - 429s 2s/step - loss: 2.4044 - accuracy: 0.3868 - val_loss: 1.1702 - val_accuracy: 0.6726
Epoch 2/200
207/207 [==============================] - 420s 2s/step - loss: 0.8306 - accuracy: 0.7722 - val_loss: 0.6342 - val_accuracy: 0.8343
Epoch 3/200
207/207 [==============================] - 421s 2s/step - loss: 0.5505 - accuracy: 0.8515 - val_loss: 0.4623 - val_accuracy: 0.8774
Epoch 4/200
207/207 [==============================] - 425s 2s/step - loss: 0.4000 - accuracy: 0.8911 - val_loss: 0.3918 - val_accuracy: 0.9006
Epoch 5/200
207/207 [==============================] - 422s 2s/step - loss: 0.3249 - accuracy: 0.9093 - val_loss: 0.3424 - val_accuracy: 0.9080
Epoch 6/200
207/207 [==============================] - 355s 2s/step - loss: 0.2846 - accuracy: 0.9206 - val_loss: 0.3085 - val_accuracy: 0.9210
Epoch 7/200
207/207 [==============================] - 335s 2s/step - loss: 0.2321 - accuracy: 0.9338 - val_loss: 0.2633 - val_accuracy:

In [15]:
loss, accuracy = model1.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model1.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9564
Testing Accuracy: 0.8917
